In [5]:
import py4dgeo
import laspy
import numpy as np
import os
import sys
sys.path.insert(0, "../src")
from fourdgeo import projection
from fourdgeo import utilities
from fourdgeo import change

## Change detection

In [ ]:
# Handle file download/reading here
# TODO: add a heibox link to the point cloud so anyone can try the script (check the notebook of the branch evolution)
infolder = r"/media/william/FA2C-06E6/weekend/low_res_bad_attr_clipped"

m3c2_settings = {"cyl_radius":1,
                 "normal_radii":[1.0,],
                 "max_distance": 10.0,
                 "registration_error":0.025
                 }

#DBScan parameters
dbscan_eps = 1
min_cluster_size = 100

In [8]:
observations = {"observations": []}

# Gather & sort only the .laz files
laz_files = sorted(f for f in os.listdir(infolder) if f.endswith('.laz'))

# Walk through each consecutive pair
for prev_fname, curr_fname in zip(laz_files, laz_files[1:]):
    prev_path = os.path.join(infolder, prev_fname)
    curr_path = os.path.join(infolder, curr_fname)

    startDateTime = utilities.iso_timestamp(prev_fname)
    endDateTime   = utilities.iso_timestamp(curr_fname)

    # Load point clouds
    epoch_0 = py4dgeo.read_from_las(prev_path)
    epoch_1 = py4dgeo.read_from_las(curr_path)

    # Compute M3C2
    m3c2 = py4dgeo.M3C2(
        epochs=(epoch_0, epoch_1),
        corepoints=epoch_0.cloud,
        cyl_radius=m3c2_settings["cyl_radius"],
        normal_radii=m3c2_settings["normal_radii"],
        max_distance=m3c2_settings["max_distance"],
        registration_error=m3c2_settings["registration_error"],
    )
    distances, uncertainties = m3c2.run()

    # Mask & stack only significant changes
    mask = np.abs(distances) >= uncertainties["lodetection"]
    if not mask.any():
        print(f"No significant changes between {prev_fname} → {curr_fname}")
        continue

    significant_pts = epoch_0.cloud[mask]
    significant_d  = distances[mask]
    changes = np.column_stack((significant_pts, significant_d))

    # Cluster & extract geoObjects
    labeled = change.cluster_m3c2_changes(changes, dbscan_eps, min_cluster_size)
    geoObjects = change.extract_geoObjects_from_clusters(labeled, endDateTime, prev_fname, curr_fname)

    observations["observations"].append({
        "backgroundImageData": {},
        "startDateTime": startDateTime,
        "endDateTime": endDateTime,
        "geoObjects": geoObjects,
    })

[2025-07-05 15:26:46][INFO] Reading point cloud from file '/media/william/FA2C-06E6/weekend/low_res_bad_attr_clipped/240825_220005.laz'
[2025-07-05 15:26:46][INFO] Reading point cloud from file '/media/william/FA2C-06E6/weekend/low_res_bad_attr_clipped/240825_230005.laz'
[2025-07-05 15:26:46][INFO] Building KDTree structure with leaf parameter 10
[2025-07-05 15:26:46][INFO] Building KDTree structure with leaf parameter 10
No clusters found between 240825_220005.laz and 240825_230005.laz.
[2025-07-05 15:26:57][INFO] Reading point cloud from file '/media/william/FA2C-06E6/weekend/low_res_bad_attr_clipped/240825_230005.laz'
[2025-07-05 15:26:57][INFO] Reading point cloud from file '/media/william/FA2C-06E6/weekend/low_res_bad_attr_clipped/240826_000005.laz'
[2025-07-05 15:26:57][INFO] Building KDTree structure with leaf parameter 10
[2025-07-05 15:26:58][INFO] Building KDTree structure with leaf parameter 10
No clusters found between 240825_230005.laz and 240826_000005.laz.
[2025-07-05 15

## Projections
### Prepare the configuration file

In [9]:
configuration = {
    "project_setting": {
        "project_name": "Rockfall_monitoring",
        "output_folder": "./out",
        "temporal_format": "%y%m%d_%H%M%S",
        "silent_mode": True,
        "include_timestamp": False
    },
    "pc_projection": {
        "pc_path": "",
        "make_range_image": True,
        "make_color_image": False,
        "top_view": False,
        "save_rot_pc": False,
        "resolution_cm": 12.5,
        "camera_position": [
            0.0,
            0.0,
            0.0
        ],
        "rgb_light_intensity": 100,
        "range_light_intensity": 10,
        "epsg": None
    }
}

### Generating the background images

In [10]:
images = []
list_background_projections = []
laz_files = sorted(f for f in os.listdir(infolder) if f.endswith('.laz'))

for enum, laz_file in enumerate(laz_files):
    pc = os.path.join(infolder, laz_file)
    lf = laspy.read(pc)
    configuration['pc_projection']['pc_path'] = pc
    background_projection = projection.PCloudProjection(
        configuration = configuration,
        project_name = configuration['project_setting']['project_name'],
        projected_image_folder = configuration['project_setting']['output_folder'],
    )
    # First projection
    if enum == 0:
        (
            ref_h_fov, ref_v_fov, ref_anchor_point_xyz, 
            ref_h_img_res, ref_v_img_res
        ) = background_projection.project_pc(buffer_m = 0.5)
    # Next projections using reference data
    else:
        background_projection.project_pc(
            ref_theta=ref_h_fov[0],
            ref_phi=ref_v_fov[0],
            ref_anchor_point_xyz=None,
            ref_h_fov=ref_h_fov,
            ref_v_fov=ref_v_fov,
            ref_h_img_res=ref_h_img_res,
            ref_v_img_res=ref_v_img_res
        )
    outfile = f"out/Rockfall_monitoring_RangeImage_{enum}_{enum+1}.tif"
    try:
        os.rename("out/Rockfall_monitoring_RangeImage.tif", outfile)
    except FileExistsError:
        os.remove(outfile)
        os.rename("out/Rockfall_monitoring_RangeImage.tif", outfile)
    images.append(outfile)

    background_projection.bg_image_filename[0] = outfile
    list_background_projections.append(background_projection)

### Projecting the change events onto the image background

In [11]:
project_name = configuration["project_setting"]["project_name"]
list_observation_projection = []

for epoch_id, observation in enumerate(observations['observations']):
    if observation['geoObjects'] is None:
        list_observation_projection.append(None)
        continue
    background_projection = list_background_projections[epoch_id]

    observation_projection = projection.ProjectChange(observation=observation,
                            project_name=f"{project_name}_{epoch_id}_{epoch_id+1}",
                            projected_image_path=background_projection.bg_image_filename[0],
                            projected_events_folder="./out",
                            epsg=None)

    observation_projection.project_change()
    list_observation_projection.append(observation_projection)

### Display the rockfall event in the study site, as soon as detected

In [12]:
from PIL import Image, ImageDraw
import json
from IPython.display import HTML

# branch_crop_box = (130, 1170, 1600, 2000) # (left, upper, right, lower)
frames = []
gif_path = "../docs/img/rockfall_projections_plus_observations.gif"

for enum, img in enumerate(images[1:]):
    frm = Image.open(img).convert("RGB")
    # frm = frm.crop(branch_crop_box)
    draw = ImageDraw.Draw(frm)

    observation_projection = list_observation_projection[enum]

    # Load geojson
    if observation_projection is not None:
        with open(observation_projection.geojson_name, 'r') as f:
            geojson_data = json.load(f)

        for feature in geojson_data["features"]:
            coords = np.array(feature["geometry"]['coordinates'][0])
            coords[:,1] *= -1
            coords = coords.reshape(len(coords)*2)
            draw.polygon(list(coords), outline='yellow', width=4)

    frames.append(frm)
    

frames[0].save(
    gif_path,
    save_all=True,
    append_images=frames[1:],
    duration=500,
    loop=0
)


# Bust cache with a unique query string
gif_display_path = gif_path + "?v=" + str(os.path.getmtime(gif_path))
HTML(f"""
<img src="{gif_display_path}" style="height:5in;" />
""")